#  Regressão Linear com Spark usando PySpark

A **regressão linear** é um algoritmo de **aprendizado de máquina supervisionado** que utiliza dados históricos para estimar o valor de algo, permitindo a previsão de eventos futuros. Existem dois tipos principais de regressão linear: simples e múltipla. A **regressão linear simples** é usada quando há apenas uma variável independente (X) para prever um resultado, enquanto a **regressão linear múltipla** é utilizada quando várias variáveis independentes (X) são necessárias para fazer a previsão.

![image.png](https://miro.medium.com/v2/resize:fit:1400/format:webp/1*kYeEvedeSMcI2ppvLSDaKw.png)


### Inicializando o PySpark

In [2]:
import findspark
import pyspark
from pyspark.sql import SparkSession

# Faz a Interafce entre o Spark e o Jupyter Notebook
findspark.init()

# Inicializando uma Sessão no Spark
spark = SparkSession.builder.appName("regresaolinear").getOrCreate()

## Hiper Parâmetros

> + **loss**: função de perda. squaredError, huber. (padrão: squaredError)
> + **maxIter**: número máximo de interações. (padrão 100)
> + **standardization**: define se os dados devem ser padronizados antes de criar o modelo. (padrão true)


---

# Aplicando Regressão Linear Multipla no DataSet Carros



## Carregando o Cunjunto de Dados Carros

In [4]:
carros_data = spark.read.csv("../Material_do_Curso/Carros.csv",
                             header=True, inferSchema=True, sep=";")
print(f"Quantidade de Registros do Dataset: {carros_data.count()}")
carros_data.show(5)

Quantidade de Registros do Dataset: 32
+-------+---------+-----------+---------------+----+-----+---------+-----------+-------+-----------+---+
|Consumo|Cilindros|Cilindradas|RelEixoTraseiro|Peso|Tempo|TipoMotor|Transmissao|Marchas|Carburadors| HP|
+-------+---------+-----------+---------------+----+-----+---------+-----------+-------+-----------+---+
|     21|        6|        160|             39| 262| 1646|        0|          1|      4|          4|110|
|     21|        6|        160|             39|2875| 1702|        0|          1|      4|          4|110|
|    228|        4|        108|            385| 232| 1861|        1|          1|      4|          1| 93|
|    214|        6|        258|            308|3215| 1944|        1|          0|      3|          1|110|
|    187|        8|        360|            315| 344| 1702|        0|          0|      3|          2|175|
+-------+---------+-----------+---------------+----+-----+---------+-----------+-------+-----------+---+
only showing top

### Escolha das Variáveis Independentes e Dependente para treinamento do Modelo

Para esse exemplo, as variáveis dependentes serão os atritutos  **Consumo | Cilindros | Cilindradas |** e a variável Dependente será o Atributo **| HP |**

ou seja, vamos usar as variáveis independentes para prever a dependente, para isso, vamos treinar o modelo com os dados do dataset.

### Importação do Módulo do PySpark Para o Pré-Processamento dos Dados

In [9]:
from pyspark.ml.feature import RFormula

In [10]:
# Colocando os dados no Formato de dataframe do Spark usando o RFormula
rformula = RFormula(formula="HP ~ Consumo + Cilindros + Cilindradas",
                    featuresCol="independentes", labelCol="dependente")
carros_rf = rformula.fit(carros_data).transform(carros_data)
carros_rf.select("independentes", "dependente").show(5, truncate=False)

+-----------------+----------+
|independentes    |dependente|
+-----------------+----------+
|[21.0,6.0,160.0] |110.0     |
|[21.0,6.0,160.0] |110.0     |
|[228.0,4.0,108.0]|93.0      |
|[214.0,6.0,258.0]|110.0     |
|[187.0,8.0,360.0]|175.0     |
+-----------------+----------+
only showing top 5 rows



### Separando os Dados de Treino e Teste

O Conjunto de dados serão separados entre treino e teste, sendo que **80%** será para treinar o modelo e **20%** para testar o modelo.

In [13]:
carros_train, carros_test = carros_rf.randomSplit([0.8, 0.2])
print(f"Quantidade de Dados de Treino: {carros_train.count()}")
print(f"Quantidade de Dados de Teste: {carros_test.count()}")

Quantidade de Dados de Treino: 22
Quantidade de Dados de Teste: 10


---

### Importação do Módulo do PySpark Para Criação do Modelo de Regressão Linear 

from pyspark.ml.regression import LinearRegression

#### Instanciando Objeto e criando o Modelo

In [17]:
# Instanciando o objeto LinearRegression
obj_rl = LinearRegression(featuresCol="independentes", labelCol="dependente",
                            maxIter=1000, loss="squaredError", standardization=True)

# Criando o Modelo
model_rl = obj_rl.fit(carros_train)

23/03/03 14:21:50 WARN Instrumentation: [700ade16] regParam is zero, which might cause numerical instability and overfitting.


#### Realizando Presição com o Modelo Criado

In [18]:
previsao = model_rl.transform(carros_test)
previsao.select("dependente", "prediction").show()

+----------+------------------+
|dependente|        prediction|
+----------+------------------+
|     110.0| 179.3068775387569|
|     205.0|220.86660273378368|
|     245.0|215.69067166446587|
|     180.0|  192.771953940586|
|     150.0|211.28883717047472|
|     180.0|188.33045377257704|
|     109.0| 77.23039291089471|
|     113.0| 51.68078231350861|
|      66.0| 48.73800627384352|
|      65.0| 46.16202464863044|
+----------+------------------+



### Avaliando o Modelo

In [19]:
from pyspark.ml.evaluation import RegressionEvaluator

In [20]:
avaliar = RegressionEvaluator(predictionCol="prediction", labelCol="dependente",
                              metricName="rmse")
rmse = avaliar.evaluate(previsao)
print(f"rmse: {rmse}")

rmse: 39.14750317480117


O **desvio quadrático médio** ou **erro quadrático médio** é uma medida frequentemente usada das diferenças entre os valores previstos por um modelo ou estimador e os valores observados. dessa forma, quanto menor for esse valor melhor o modelo.

Para o modelo avaliado nesse exemplo, podemos observar que esse valor é relativamente grande, isso ocorre por conta do tamanho do conjunto de dados que é pequeno.